In [ ]:
!pip install timm torch torchvision --quiet

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast # <<< Import for mixed precision

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Corrected for 3 channels
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Corrected for 3 channels
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2, pin_memory=True) # Increased batch size
testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
model = timm.create_model(
    'resnet18',
    pretrained=False,
    num_classes=10
).to(device)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2) # Adjusted LR
criterion = nn.CrossEntropyLoss()
scaler = GradScaler() # <<< CHANGE 2: Create a gradient scaler

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    loop = tqdm(trainloader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        # <<< CHANGE 3: Use autocast for mixed precision
        with autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        # <<< CHANGE 4: Scale loss and update weights
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Update tqdm description
        loop.set_postfix(loss=loss.item(), acc=f"{100*correct/total:.2f}%")


    print(f"Epoch {epoch+1} Summary | Train Loss: {train_loss/len(trainloader):.4f} | Train Acc: {100*correct/total:.2f}%")


/tmp/ipython-input-4172094914.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() # <<< CHANGE 2: Create a gradient scaler
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/20:   0%|          | 0/196 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipython-input-4172094914.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not avail

Epoch 1 Summary | Train Loss: 1.6743 | Train Acc: 37.86%


Epoch 2/20: 100%|██████████| 196/196 [12:05<00:00,  3.70s/it, acc=54.14%, loss=1.1]


Epoch 2 Summary | Train Loss: 1.2586 | Train Acc: 54.14%


Epoch 3/20: 100%|██████████| 196/196 [12:19<00:00,  3.77s/it, acc=61.24%, loss=0.985]


...

Epoch 18 Summary | Train Loss: 0.4993 | Train Acc: 82.33%


Epoch 19/20: 100%|██████████| 196/196 [12:12<00:00,  3.74s/it, acc=83.19%, loss=0.354]


Epoch 19 Summary | Train Loss: 0.4790 | Train Acc: 83.19%


Epoch 20/20: 100%|██████████| 196/196 [11:53<00:00,  3.64s/it, acc=83.62%, loss=0.442]

Epoch 20 Summary | Train Loss: 0.4675 | Train Acc: 83.62%


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in testloader:
        imgs, labels = imgs.to(device), labels.to(device)
        with autocast(): # Use autocast during inference as well
            outputs = model(imgs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"\n✅ Final Test Accuracy: {100*correct/total:.2f}%")

/tmp/ipython-input-537418886.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Use autocast during inference as well



✅ Final Test Accuracy: 79.48%


In [ ]:
from torch.optim import lr_scheduler

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

for _ in range(20):
    scheduler.step()

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


In [ ]:


from torch.optim import lr_scheduler

print("Creating the learning rate scheduler...")
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)
print("Catching scheduler up to epoch 20...")
for _ in range(20):
    scheduler.step()

ADDITIONAL_EPOCHS = 20
STARTING_EPOCH = 20
print(f"\nContinuing training for {ADDITIONAL_EPOCHS} more epochs...")

for epoch in range(STARTING_EPOCH, STARTING_EPOCH + ADDITIONAL_EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    loop = tqdm(trainloader, desc=f"Epoch {epoch+1}/{STARTING_EPOCH + ADDITIONAL_EPOCHS}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=f"{100*correct/total:.2f}%")

    print(f"Epoch {epoch+1} Summary | Train Loss: {train_loss/len(trainloader):.4f} | Train Acc: {100*correct/total:.2f}%")

    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.amp.autocast(device_type="cuda"):
                outputs = model(imgs)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    print(f"✅ Epoch {epoch+1} Test Accuracy: {100*test_correct/test_total:.2f}%")

    scheduler.step()

print(f"\n✅ Final Test Accuracy after {STARTING_EPOCH + ADDITIONAL_EPOCHS} total epochs: {100*test_correct/test_total:.2f}%")

Creating the learning rate scheduler...
Catching scheduler up to epoch 20...

Continuing training for 20 more epochs...


Epoch 21/40: 100%|██████████| 196/196 [13:14<00:00,  4.05s/it, acc=87.05%, loss=0.525]

Epoch 21 Summary | Train Loss: 0.3659 | Train Acc: 87.05%


✅ Epoch 21 Test Accuracy: 82.29%


Epoch 22/40: 100%|██████████| 196/196 [11:26<00:00,  3.50s/it, acc=88.25%, loss=0.194]

...

Epoch 39 Summary | Train Loss: 0.2426 | Train Acc: 91.53%


✅ Epoch 39 Test Accuracy: 83.99%


Epoch 40/40: 100%|██████████| 196/196 [12:17<00:00,  3.76s/it, acc=91.46%, loss=0.236]

Epoch 40 Summary | Train Loss: 0.2422 | Train Acc: 91.46%


✅ Epoch 40 Test Accuracy: 83.87%

✅ Final Test Accuracy after 40 total epochs: 83.87%
